<a href="https://colab.research.google.com/github/MelissaPerez09/Proyecto03-CC3086/blob/main/Proyecto_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade gspread

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gspread
    Found existing installation: gspread 3.4.2
    Uninstalling gspread-3.4.2:
      Successfully uninstalled gspread-3.4.2


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

wb = gc.open_by_key('1Is-380wsfiKOzSR7p-TWEQkPFeUlXViq0xc1rFMH_GU')
ws = wb.worksheet('datos')

# get_all_values gives a list of rows.
rows = ws.get_all_values()

# Convert to a DataFrame and render.
import pandas as pd
df = pd.DataFrame.from_records(rows)
print(df)


               0        1        2 3                        4
0    Temperatura  presion  Altitud     ID de la celda proxima
1           28.3   852.04  1437.61                        140
2          28.37    852.1  1437.91                           
3          28.43   852.03  1438.29                           
4          31.62   852.45  1433.12                           
..           ...      ...      ... ..                     ...
136        29.12   862.77   1327.1                           
137        28.85   857.22   1388.4                           
138        22.17   854.87  1410.73                           
139        22.17   854.82   1411.0                           
140        26.14   854.83  1409.95                           

[141 rows x 5 columns]


In [ ]:
from google.colab import auth
auth.authenticate_user()

import numpy as np
from numba import cuda
import gspread
import math
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

wb = gc.open_by_key('1Is-380wsfiKOzSR7p-TWEQkPFeUlXViq0xc1rFMH_GU')
ws = wb.worksheet('DatosTransformar')

# get_all_values gives a list of rows.
rows = ws.get_all_values()

@cuda.jit
def celcius_kelvin(y,out):
  indx = cuda.grid(1)
  out[indx] = y[indx] + 273.15

#main
array = np.array(ws.get_all_values())
arrayFloat = []
for i in range(array.size):
  arrayFloat.append(float(array[i][0]))

# Traslado de memoria de host a device
d_a = cuda.to_device(arrayFloat)
# creamos la salida en el device
d_out = cuda.device_array_like(d_a)

# usamos formula para sacar los bloques, cada uno con 256 threads
N = 10000
threads_per_block = 256
blocks_per_grid = (N // threads_per_block) + 1
celcius_kelvin[blocks_per_grid, threads_per_block](d_a, d_out)
# wait for all threads to complete
cuda.synchronize()
# copia de informacion de device a host
# e impresion de ello
h_out = d_out.copy_to_host()
for i in range(h_out.size):
  print(str(round(h_out[i],2))+"K")


301.45K
301.52K
301.58K
304.77K
306.24K
306.78K
307.12K
307.39K
307.8K
307.92K
308.01K
308.08K
308.15K
308.19K
308.22K
308.32K
302.03K
302.12K
300.06K
300.06K
300.06K
300.06K
300.21K
302.38K
303.22K
303.84K
304.17K
304.2K
303.8K
301.06K
300.99K
300.98K
300.94K
301.18K
303.06K
303.46K
303.73K
303.98K
304.23K
304.4K
294.65K
294.64K
294.62K
294.63K
299.44K
299.44K
299.43K
299.4K
299.4K
299.47K
299.46K
299.45K
299.42K
299.49K
299.5K
299.51K
299.5K
299.49K
299.48K
299.46K
302.44K
303.64K
304.17K
304.49K
304.58K
299.55K
299.54K
299.55K
299.54K
299.54K
299.53K
299.49K
299.49K
299.5K
299.5K
299.54K
299.49K
299.48K
299.47K
299.45K
299.42K
299.41K
299.4K
299.42K
299.4K
299.37K
299.37K
299.35K
299.33K
299.27K
299.25K
299.21K
299.21K
299.17K
299.18K
299.18K
299.19K
299.2K
299.2K
299.19K
299.2K
299.21K
299.21K
299.2K
299.2K
299.17K
299.18K
299.16K
299.15K
299.14K
299.13K
299.15K
299.13K
299.15K
299.3K
299.28K
299.27K
299.27K
299.27K
299.27K
299.28K
299.24K
299.25K
299.24K
299.26K
299.24K
299.23K
29

/usr/local/lib/python3.7/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 40 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
